You need to implement a function that determines the status of a board game based on a list of moves and the size of the board.

Your Task: implement a function that runs through the list of moves and return one of the following game statuses:

"in progress"
"player 1 is the winner"
"player 2 is the winner"
Assume that all moves are valid and well formed. Some tests are provided, which you can run as often as you'd like.

Rules
Board Setup: The board is always a square (e.g., 3x3, 6x6). It is between 3x3 and 9x9 in size.

Turn Actions: Each player’s turn consists of two types of actions, in this order:
Place: The player must add a piece to the board once.
Topple: Zero or more times, a player takes pieces from a square they own and places them in one direction.
Actions work as follows:

Place:
A piece may only be placed on an open square, or a square with that player’s pieces.
If the square is open, a piece is put on that square and that player owns that square.
If there are already pieces on the square, another is added.

Topple:
First, the player picks up all the pieces from a square of 2 or more, then places them one at a time in a particular direction (left/right/up/down) starting with the next adjacent square.
If any of the placed pieces land on an opponent's square, those pieces are captured and become theirs.
Pieces that “fall off” the end of the game board are lost.

Game End
The game ends when all of one player's pieces are captured.

Input Format
You will receive the list of moves as an array of strings, where each string is a move.

Place moves

A Place move is denoted by a "p" followed by the coordinates:
"p22" would mean a piece was placed at the coordinates 2,2

Topple moves
A Topple move is denoted by a "t", the coordinates, and then a direction which is one of "u", "r", "d", "l" corresponding to up, right, down and left:

"t00r" would mean the pieces at the coordinates 0,0 topple to the right.
If there were say 3 pieces on the square 0,0 these pieces would end up occupying the coordinates 0,1 | 0,2 | 0,3

["p10", "p12", "p10", "t10r"] player 1 is the winner
["p00", "p22", "p02", "p22", "t22u", "t02l"] player 2 is the winner

when place is encountered, switch the player

In [ ]:
# My Solution

class Game:
    def __init__(self, grid_size = 3) -> None:
        # 0 = empty, 1 = player 1 tile, 2 = player 2 tile
        self.grid_size = grid_size
        self.board = [[0 for _ in range(grid_size)] for _ in range(grid_size)]
        self.p1_count = 0
        self.p2_count = 0
        self.winner = None
        self.game_over = False
    
    def display(self):
        for row in range(self.grid_size):
            for col in range(self.grid_size):
                print(f"{self.board[row][col]:2}", end=" ")
            print()
                
                
    
    def place(self):
        pass
    
    def topple(self):
        pass

def main():
    print("START GAME")
    
    # Get board size
    while True:
        s = input("Enter the grid size between 3 and 9 [3]: ").strip()
        if s == "":
            grid_size = 3
            break
        try:
            grid_size = int(s)
        except ValueError:
            print("Invalid input. Please enter an integer between 3 and 9.")
            continue
        if 3 <= grid_size <= 9:
            break
        print("Number out of range. Please enter a value from 3 to 9.")
    
    # Initialize the board
    game = Game(grid_size)
    
    while not game.game_over:
        pass
    
    # Display winner
    game.display()

main()

In [ ]:
# Gemini Initial Solution

class ToppleGame:
    def __init__(self, board_size=3):
        # The board is a dictionary where keys are (row, col) tuples.
        # Values are dictionaries holding the 'owner' (1 or 2) and 'count' of pieces.
        self.board_size = board_size
        self.board = {}
        self.current_player = 0 
        self.winner = None
        self.game_active = True

    def _is_on_board(self, r, c):
        """Helper to check if coordinates are within the board boundaries."""
        return 0 <= r < self.board_size and 0 <= c < self.board_size

    def _get_piece_counts(self):
        """Returns a tuple (player_1_pieces, player_2_pieces) currently on the board."""
        p1_count = 0
        p2_count = 0
        for cell in self.board.values():
            if cell['owner'] == 1:
                p1_count += cell['count']
            elif cell['owner'] == 2:
                p2_count += cell['count']
        return p1_count, p2_count

    def play_moves(self, moves):
        """Processes a list of moves and determines the winner."""
        for move in moves:
            if not self.game_active:
                break

            action = move[0]
            r = int(move[1])
            c = int(move[2])

            if action == 'p':
                self._place(r, c)
            elif action == 't':
                direction = move[3]
                self._topple(r, c, direction)

            # Check for a winner after every move.
            # We only check if both players have actually entered the game
            # (to prevent Player 2 from "losing" before they even place a piece).
            p1_count, p2_count = self._get_piece_counts()
            if self.current_player == 1 and p2_count == 0 and p1_count > 0 and len(moves) > 1:
                 # Check if P2 has played at least once. If P2 is wiped out, P1 wins.
                 # Note: A stricter implementation might track if P2 has placed their first piece yet.
                 self.winner = 1
                 self.game_active = False
            elif self.current_player == 2 and p1_count == 0 and p2_count > 0:
                 self.winner = 2
                 self.game_active = False

        if self.winner:
            return f"Player {self.winner} is the winner"
        else:
            return "Game is still ongoing"

    def _place(self, r, c):
        # "when place is encountered, switch the player"
        self.current_player = 2 if self.current_player == 1 else 1

        if (r, c) not in self.board:
            self.board[(r, c)] = {'owner': self.current_player, 'count': 1}
        else:
            # Rule: "If there are already pieces on the square, another is added."
            if self.board[(r, c)]['owner'] == self.current_player:
                self.board[(r, c)]['count'] += 1

    def _topple(self, r, c, direction):
        # Validate square belongs to current player and has 2 or more pieces
        if (r, c) not in self.board or self.board[(r, c)]['owner'] != self.current_player:
            return
        if self.board[(r, c)]['count'] < 2:
            return

        # Pick up all pieces
        pieces_to_distribute = self.board[(r, c)]['count']
        del self.board[(r, c)] # Square becomes empty

        # Set direction modifiers
        dr, dc = 0, 0
        if direction == 'u': dr = -1
        elif direction == 'd': dr = 1
        elif direction == 'l': dc = -1
        elif direction == 'r': dc = 1

        curr_r, curr_c = r, c

        # Distribute pieces one by one
        while pieces_to_distribute > 0:
            curr_r += dr
            curr_c += dc
            pieces_to_distribute -= 1

            # If the piece falls off the board, it is lost. 
            # We simply don't place it and continue the loop to discard the rest.
            if not self._is_on_board(curr_r, curr_c):
                continue 

            # Place the piece on the board
            if (curr_r, curr_c) not in self.board:
                self.board[(curr_r, curr_c)] = {'owner': self.current_player, 'count': 1}
            else:
                # If landing on opponent's square, capture it! 
                # If landing on own square, just add to it.
                self.board[(curr_r, curr_c)]['owner'] = self.current_player
                self.board[(curr_r, curr_c)]['count'] += 1

# --- Test Cases ---

# Example 1
game1 = ToppleGame(board_size=3)
moves1 = ["p10", "p12", "p10", "t10r"]
print(f"Moves 1 Result: {game1.play_moves(moves1)}") 
# Expected: Player 1 is the winner

# Example 2
game2 = ToppleGame(board_size=3)
moves2 = ["p00", "p22", "p02", "p22", "t22u", "t02l"]
print(f"Moves 2 Result: {game2.play_moves(moves2)}") 
# Expected: Player 2 is the winner

In [ ]:
# Gemini Single Fuction Solution

def get_game_status(moves, board_size):
    # Dictionary to hold the board state: {(row, col): {'owner': int, 'count': int}}
    board = {}
    current_player = 0
    p2_has_played = False
    
    for move in moves:
        action = move[0]
        r = int(move[1])
        c = int(move[2])
        
        if action == 'p':
            # "when place is encountered, switch the player"
            current_player = 2 if current_player == 1 else 1
            if current_player == 2:
                p2_has_played = True
                
            if (r, c) not in board:
                board[(r, c)] = {'owner': current_player, 'count': 1}
            else:
                board[(r, c)]['count'] += 1
                board[(r, c)]['owner'] = current_player 
                
        elif action == 't':
            direction = move[3]
            
            # Topple only if the square exists and has 2 or more pieces
            if (r, c) in board and board[(r, c)]['count'] >= 2:
                pieces_to_distribute = board[(r, c)]['count']
                del board[(r, c)] # Pick up all pieces
                
                dr, dc = 0, 0
                if direction == 'u': dr = -1
                elif direction == 'd': dr = 1
                elif direction == 'l': dc = -1
                elif direction == 'r': dc = 1
                
                curr_r, curr_c = r, c
                
                while pieces_to_distribute > 0:
                    curr_r += dr
                    curr_c += dc
                    pieces_to_distribute -= 1
                    
                    # Check if the piece is still on the board
                    if 0 <= curr_r < board_size and 0 <= curr_c < board_size:
                        if (curr_r, curr_c) not in board:
                            board[(curr_r, curr_c)] = {'owner': current_player, 'count': 1}
                        else:
                            # Add piece and take ownership (capturing if it was the opponent's)
                            board[(curr_r, curr_c)]['owner'] = current_player
                            board[(curr_r, curr_c)]['count'] += 1
                    else:
                        # If a piece falls off, the rest of the stack will fall off too
                        break
        
        # Check Win Condition after the move
        # We only check if Player 2 has actually entered the game to avoid a premature P1 win
        if p2_has_played:
            p1_count = sum(1 for cell in board.values() if cell['owner'] == 1)
            p2_count = sum(1 for cell in board.values() if cell['owner'] == 2)
            
            if p2_count == 0 and p1_count > 0:
                return "player 1 is the winner"
            elif p1_count == 0 and p2_count > 0:
                return "player 2 is the winner"

    # If the loop finishes without returning a winner
    return "in progress"

# --- Tests ---
print(get_game_status(["p10", "p12", "p10", "t10r"], 3)) 
# Output: "player 1 is the winner"

print(get_game_status(["p00", "p22", "p02", "p22", "t22u", "t02l"], 3)) 
# Output: "player 2 is the winner"

Since this is an OOP design challenge, the goal is to create a clean, maintainable structure where each class has a clear responsibility. We will define a Game class to manage the high-level logic and state, and a Board or Cell structure to handle piece management.
OOP Design Strategy

    The Player: We track the current player and a flag to see if Player 2 has started, ensuring the win condition is only checked once the game is truly in motion.

    The Board: Represented as a dictionary of Cell objects (or simple dictionaries) to handle the 3x3 to 9x9 grid dynamically.

    The Actions: Encapsulate place and topple as methods that modify the board state.

    Capture Logic: Centralize the ownership change logic within the distribution loop to ensure toppling pieces always convert opponent squares.

In [ ]:
# Gemini OOP Solution

class ToppleGame:
    def __init__(self, board_size=3):
        if not (3 <= board_size <= 9):
            raise ValueError("Board size must be between 3 and 9.")
            
        self.size = board_size
        self.board = {} # Keys: (r, c), Values: {'owner': int, 'count': int}
        self.current_player = 2 
        self.p2_has_started = False

    def _is_on_board(self, r, c):
        return 0 <= r < self.size and 0 <= c < self.size

    def simple_render(self):
        for r in range(self.size):
            row = []
            for c in range(self.size):
                cell = self.board.get((r, c))
                # Format: P1.3 (Player 1, 3 pieces) or " . " if empty
                row.append(f"P{cell['owner']}.{cell['count']}" if cell else " . ")
            print(" ".join(row))
        print("-" * (self.size * 4)) # Visual separator
    
    # def render(self):
    #     """Prints the current board state. Call this manually to debug."""
    #     print("\n   " + " ".join([f" {c}  " for c in range(self.size)]))
    #     print("  +" + "----+" * self.size)
    #     for r in range(self.size):
    #         row_str = f"{r} |"
    #         for c in range(self.size):
    #             cell = self.board.get((r, c))
    #             row_str += f"P{cell['owner']}:{cell['count']}|" if cell else "    |"
    #         print(row_str)
    #         print("  +" + "----+" * self.size)

    def place(self, r, c):
        # Toggles the current_player to the other player
        self.current_player = 1 if self.current_player == 2 else 2
        
        if self.current_player == 2:
            self.p2_has_started = True
            
        if self._is_on_board(r, c):
            # If the spot on the board is empty
            if (r, c) not in self.board:
                self.board[(r, c)] = {'owner': self.current_player, 'count': 1}
            else:
                self.board[(r, c)]['count'] += 1
                self.board[(r, c)]['owner'] = self.current_player

    def topple(self, r, c, direction):
        # If the cell in the board in empty OR the owner of the cell is not the current player we cannot topple it
        if (r, c) not in self.board or self.board[(r, c)]['owner'] != self.current_player:
            return
        
        # If there are less than 2 pieces on this cell we cannot topple it
        if self.board[(r, c)]['count'] < 2:
            return

        pieces = self.board[(r, c)]['count']
        # Empty the cell that is to be toppled
        del self.board[(r, c)]

        vectors = {'u': (-1, 0), 'd': (1, 0), 'l': (0, -1), 'r': (0, 1)}
        dr, dc = vectors[direction]

        curr_r, curr_c = r, c
        for _ in range(pieces):
            curr_r += dr
            curr_c += dc
            if self._is_on_board(curr_r, curr_c):
                # If the cell is empty
                if (curr_r, curr_c) not in self.board:
                    self.board[(curr_r, curr_c)] = {'owner': self.current_player, 'count': 1}
                else:
                    self.board[(curr_r, curr_c)]['owner'] = self.current_player
                    self.board[(curr_r, curr_c)]['count'] += 1
            else:
                break

    # There is a more efficient alt approach The "Counter" Approach, you would maintain a running piece count as an attribute of the class
    # Update: Every time you place, topple, or capture, you increment/decrement these counters
    # Then the The get_status method becomes O(1), but this is more complex and Maintaining counters across complex "Topple" distributions is prone to "Off-by-One" errors
    # Stick to this scanning approach for interview
    def get_status(self):
        p1_any = False
        p2_any = False
        for cell in self.board.values():
            if cell['owner'] == 1:
                p1_any = True
            
            if cell['owner'] == 2:
                p2_any = True
            
            if p1_any and p2_any:
                break
            
        # If Player 2 hasn't even placed their first piece yet, the game is "in progress" by default, even if they have 0 pieces on the board.
        if self.p2_has_started:
            if p1_any and not p2_any: return "player 1 is the winner"
            if p2_any and not p1_any: return "player 2 is the winner"
        
        return "in progress"

def solve(moves, board_size=3):
    game = ToppleGame(board_size)
    
    for move in moves:
        action = move[0]
        r, c = int(move[1]), int(move[2])
        
        if action == 'p':
            game.place(r, c)
        elif action == 't':
            game.topple(r, c, move[3])
            
        status = game.get_status()
        if status != "in progress":
            game.simple_render() # Uncomment this line if you want to debug the winning state
            return status
            
    # game.render() # Uncomment this line to see the board at the end of an 'in progress' game
    return "in progress"

# --- Verification ---
print(solve(["p10", "p12", "p10", "t10r"], 3)) # player 1 is the winner
print(solve(["p00", "p22", "p02", "p22", "t22u", "t02l"], 3)) # player 2 is the winner
print(solve(["p00", "p22", "p00"], 3)) # in progress